# Correlation Network

Correlation network (graph) desribes the strength of correlation among qubit pairs in a system and shows how qubits in a system is correlated. 

In the network:

Nodes: represent the *qubits*.

Edge: represent *how much* the nodes (qubits) are *correlated* by its weight.

The strength can be any value that quantify how a qubit pair is correlated. In previous studies, quantum Mutual Information (MI) is used. Though it remains unknown, we conjecture that Entanglement of Formation (EoF) and various entanglement witnesses are also useful quantities in certain scenario. 

In quantum computing, people can calculate the correlation network of a objective wavefunction, such as the network of the ground state of a Hamiltonian in VQE, in a approximated way, before carrying out quantum computing on the system. 

By this pre-calculation, people can know that

- Which qubits are more active and which qubits are nearly stationary and can be removed.
- The groups of qubits in which the qubits are inter-entangled, so that the computation can be divided into parts.
- Which part of the system is more active and needs higher precision of operation. 
    - So that physics qubits with higher quality can be assigned to it.

We believe that the potential applications shown above make pre-calculation of correlation network very important for near-term quantum computing because

- The number of gates and qubits allowed on near-term quantum devices is very limited.
- The quality of qubits in near-term is not uniform and full-connectivity qubits set will be small.

To address these problems, we provide modules for

- Easily carrying out correlation network approximation from qubit Hamiltonians.
- Finding the optimal mapping to physical qubits based on the correlation network.
- Finding the subsets of qubits which are most correlated.
- Finding the community (correlated group) structure of a correlation network.

## Classical pre-calculation

The classical pre-calculation can be easily carried out as following.

In [1]:
from Precalculation.iTensorCore import run_classcal_precalculation
from HamiltonianGenerator import make_example_LiH
from HamiltonianGenerator.FermionTransform import jordan_wigner

# Generate the problem Hamiltonian
energy_obj = make_example_LiH(fermi_qubit_transform=jordan_wigner)

# Run the classical pre-calculation
classical_res=run_classcal_precalculation(energy_obj.n_qubit,energy_obj.hamiltonian,calc_2DM=True)
print("Energy",classical_res["energy"])
print("Entropy",classical_res["entropy"])

ModuleNotFoundError: No module named 'dmrgpy'

Because `calc_2DM` is made `True`, reduced two-qubit density matrices of the ground state is also calculated. Here we show how to use them calculated the correlation quantities.

In [ ]:
from Utilities.WaveLocalProperties import get_mutual_information_by_2DMs,get_EoF_by_2DMs
 
classical_res["MI"]=get_mutual_information_by_2DMs(classical_res["2DM"])
classical_res["EoF"]=get_EoF_by_2DMs(classical_res["2DM"])

print("Mutual Information",classical_res["MI"].tolist())
print("Entanglement of Formation",classical_res["EoF"].tolist())

We can convert the weight matrix to a network and visulize it as following.

In [2]:
from Network import get_nx_graph_by_adjacent_mat, draw_graph

G = get_nx_graph_by_adjacent_mat(classical_res["MI"])
draw_graph(G)

NameError: name 'classical_res' is not defined

## Most correlated subsets

In [ ]:
from Network.GA._ga_selector import GACorrelationQsubsetSelector

G = get_nx_graph_by_adjacent_mat(classical_res["MI"])
selector = GACorrelationQsubsetSelector(G)

# Run GA selector with time budget 60 seconds
selector.run(time_budget=60)

# Get Result

results = selector.get_result()

print(results)


## Community detection

In [ ]:
from Network._community_detection import detect_nx_graph_community, draw_community_graph
from Network import get_nx_graph_by_adjacent_mat

# Initial graph
G = get_nx_graph_by_adjacent_mat(classical_res["MI"])

# Detect community
community_map = detect_nx_graph_community(G)

# Draw community graph
draw_community_graph(G, community_map, "output")




## Optimal Mapping

Optimal Mapping (graph) desribes the graph embedding of source graph in target graph.

In the network:

Nodes: represent the *qubits*.

Edge: represent *mutual information*.

We feed the generic constructor with both source weighted graph and target weighted graph. The optimal results will be evolved and optimized though a heuristic and generic hybrid algorithm.

In [ ]:
from Network.GA_Graph_Embedding._ga_constructor import GAConstructor
from Network._quantum_chips import *


# Initial quantum chips
rigetti_16Q_Aspen = Rigetti_16Q_Aspen()
ibm_20Q_Johannesburg = IBM_20Q_Johannesburg()

# Search optimal mapping
embeding_selector = GAConstructor(rigetti_16Q_Aspen, ibm_20Q_Johannesburg)
embeding_selector.run(time_budget = 60)

# Show optimal mapping
results = embeding_selector.get_results()
print(results)